In [1]:
import pandas as pd
import numpy as np
from gretel_synthetics.timeseries_dgan.dgan import DGAN
from gretel_synthetics.timeseries_dgan.config import DGANConfig
from gretel_synthetics.timeseries_dgan.config import  OutputType
import os
from os import path
from sklearn.preprocessing import MinMaxScaler
from tsl.datasets import AirQuality, MetrLA, PemsBay

dataset_name = 'AirQuality'
df = AirQuality(small=True)

In [2]:
# Create some random training data
df = df.dataframe()[-2000:]
df.columns = df.columns.droplevel('channels')
df_cols = df.columns
scaler = MinMaxScaler()
df = scaler.fit_transform(df)
df = pd.DataFrame(df)
df.columns = df_cols

In [3]:
# Train the model
model_path = f'models/{dataset_name}.model'

if path.exists(model_path):
    model = DGAN.load(model_path)
else:
    model = DGAN(DGANConfig(
        max_sequence_len=len(df_cols),
        sample_len=6,
        batch_size=100,
        epochs=50,  # For real data sets, 100–1000 epochs is typical
    ))

    model.train_dataframe(
        df,
    )
    
    model.save(model_path)
# Generate synthetic data
synthetic_df = model.generate_dataframe(1000)
synthetic_df = scaler.inverse_transform(synthetic_df)
synthetic_df = pd.DataFrame(synthetic_df)
synthetic_df.columns = df.columns

/Users/paulmagos/.pyenv/versions/3.11.9/envs/Dopple/lib/python3.11/site-packages/gretel_synthetics/timeseries_dgan/dgan.py:688: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=self.config.mixed_precision_training)
2024-09-15 18:23:59,166 : MainThread : INFO : epoch: 0
/Users/paulmagos/.pyenv/versions/3.11.9/envs/Dopple/lib/python3.11/site-packages/gretel_synthetics/timeseries_dgan/dgan.py:696: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(
/Users/paulmagos/.pyenv/versions/3.11.9/envs/Dopple/lib/python3.11/site-packages/gretel_synthetics/timeseries_dgan/dgan.py:719: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):
/Users/paulmagos/.pyenv/ver

In [4]:
synthetic_df.to_csv(f'{os.getcwd()}/../../Datasets/GeneratedDatasets/{dataset_name}/syntetic{dataset_name}DGAN.csv', index=False)